# LangChain with HuggingFace and OpenAI

In [1]:
import os

from langchain import (
    LLMChain,
    PromptTemplate,
)

## Set the template

In [2]:
template = """How do you say '{phrase}' in {language}?
"""

prompt = PromptTemplate(template=template,
                        input_variables=['phrase', 'language'])


In [17]:
phrase = 'I love brownies'
language = 'Spanish'

prompt.format(phrase=phrase, language=language)

"How do you say 'I love brownies' in Spanish?\n"

## LLMChain with HuggingFace models

In [4]:
from langchain import HuggingFaceHub

In [5]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.environ.get('HF_ACCESS_TOKEN')

In [6]:
hub_llm = HuggingFaceHub(
    repo_id='google/flan-t5-base',
    model_kwargs={'temperature': 1e-10}
)

llm_chain = LLMChain(
    llm=hub_llm,
    prompt=prompt,
)

In [18]:
llm_chain.run({'phrase': phrase, 'language': language})

'\n"Me encantan los brownies"'

## LLMChain with OpenAI models

In [8]:
from langchain import OpenAI

In [9]:
api_key = os.environ.get('OPENAI_API_KEY')

In [19]:
openai_llm = OpenAI(
    temperature=0,
    openai_api_key=api_key,
    model_name="text-davinci-003"
)

llm_chain = LLMChain(
    llm=openai_llm,
    prompt=prompt,
)

In [20]:
llm_chain.run({'phrase': phrase, 'language': language})

'\n"Me encantan los brownies"'

## LLMChain with Conversational Memory

In [12]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

In [21]:
openai_llm = OpenAI(
    temperature=0,
    openai_api_key=api_key,
    model_name="text-davinci-003"
)

conversation_buf = ConversationChain(
    llm=openai_llm,
    memory=ConversationBufferMemory()
)

In [22]:
# Helper
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [23]:
count_tokens(
    conversation_buf, 
    "What are possible reasons why my brownies are dry"
)

Spent a total of 128 tokens


" That's a great question! It could be due to a few different reasons. It could be that you didn't use enough butter or oil, or that you overmixed the batter. It could also be that you cooked them for too long or at too high of a temperature."

In [24]:
count_tokens(
    conversation_buf, 
    "But what type of butter do I need?"
)

Spent a total of 178 tokens


" The type of butter you need depends on the recipe you're using. Generally, you'll want to use unsalted butter, but some recipes may call for salted butter."

In [25]:
count_tokens(
    conversation_buf, 
    "What are we talking about again?"
)

Spent a total of 205 tokens


" We're talking about possible reasons why your brownies are dry."